# description


Une analyse des différents indicateurs de vente, EDA, Segmentation


1- Nettoyage et de préparation des données :
Validation des données : Assurez-vous que les colonnes, types de données et valeurs respectent les contraintes du schéma cible.
Détection des anomalies : Détecter et traiter les incohérences dans les fichiers produits, ventes, clients (ex : valeurs erronées, valeurs aberrantes, produits vendus mais non présents dans la table products, anomalies dans les prix, etc.).    Transformation des données : Appliquer les transformations nécessaires (ex : mapping des sous catégories, ajout de nvelles colonnes, etc ).

2- Prendre en compte la nouvelle colonne sub_category dans le schéma de données du datamart des ventes et mettre à jour le pipeline ETL pour que la nouvelle sous-catégorie soit incluse lors du chargement des données de la table des ventes.

3- faire une EDA (analyse exploratoire) et vérifier /détecter la présence d’anomalies potentielles .

4- Tester et valider les modifications pour s’assurer que les données sont correctement intégrées. 

# Imports

In [ ]:
import os
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
from pathlib import Path
import plotly.io as pio
from datetime import datetime

In [ ]:
path=Path('./data')
os.listdir(path)

In [ ]:
#todo add dtype default dict for better performance
cust=pd.read_csv(path/'clients.csv',sep=";", encoding="utf8")
products=pd.read_csv(path/'produits_sous-categorie.csv',sep=",", encoding="utf8")
tran=pd.read_csv(path/'ventes.csv',sep=",", encoding="utf8")

In [ ]:
#set all dataframe into a dict
ddf={'customs':cust,"products":products,"transaction":tran}

In [ ]:
#shape of each dataframe
for k,v in ddf.items():
    print(f'{k}')
    print(f'{v.shape}\n')

In [ ]:
#info (dtype) of each dataframe
for k,v in ddf.items():
    print(f'{k}')
    print(f'{v.info()}\n')

In [ ]:
# describe on each dataframe
for k,v in ddf.items():
    print(f'{k}')
    print(f"{v.describe(include='all')}\n")

In [ ]:
# sample or head of each dataframe
for k,v in ddf.items():
    print(f'{k}')
    print(f'{v.head(5)}\n')

In [ ]:
for k,v in ddf.items():
    print(f'Pecentage of Nan values in table: {k} \nby column:"\n{v.isnull().sum()}\n')

# Univariate analysis and cleaning

## clients

In [ ]:
print(cust.duplicated().sum())

In [ ]:
cust.sample(10)

In [ ]:
data = cust['birth'].values

mean = np.mean(data)
std_dev = np.std(data)

z_scores = stats.zscore(data)

outliers = np.where(np.abs(z_scores) > 3)

print(f"Mean: {mean:.2f}")
print(f"Standard Deviation: {std_dev:.2f}")
print(f"Number of outliers (beyond 3-sigma): {len(outliers[0])}")
print(f"Outlier values: {data[outliers]}")

In [ ]:
#drop the outlier
cust.drop(cust.index[outliers[0]], inplace=True)

In [ ]:
title="The breakdown of customers by gender and birth year"
fig = px.histogram(cust,x='birth',color='sex',marginal='box',text_auto=True,title=title)
# pio.write_image(fig,Path(path/title),format='png')
fig.show()

In [ ]:
# recall on cust info before jump into next dataframe
cust.info()

In [ ]:
tmp=cust.sex.value_counts()
fig=px.pie(values=tmp.values)
fig.show()

## products

In [ ]:
#no duplicates in the product id
print(products.duplicated('product_id').sum())

In [ ]:
products.head(1)

In [ ]:
#as see above, there is negative price, which has T_0 value, test product_id obviously
products[products['price']==-1]

#### check diff category vs sub-category

In [ ]:
title='Products table items by price and category'
fig=px.histogram(products, x='price',color='category',marginal='box',title=title)
fig.show()

In [ ]:
title='The breakdown of products by price and sub category'
fig=px.histogram(products, x='price',color='sub_category',marginal='box',title=title)
#pio.write_image(fig,Path(path/title),format='png')
fig.show()

In [ ]:
products.groupby('category').agg(nunique=('sub_category', 'nunique'))

In [ ]:
products[['category','sub_category']].drop_duplicates()

In [ ]:
#recall on info then next dataframe
products.info()

## transactions

In [ ]:
tran[tran['id_prod']=="T_0"]

In [ ]:
#let clean test data (as seen in descrbe results, product_id=T_0 or date startwith test)
def is_correct_format(dt_string):
    try:
        datetime.strptime(dt_string, "%Y-%m-%d %H:%M:%S.%f")
        return True
    except ValueError:
        return False

# Apply the function to the column
tran['date_format']= tran['date'].apply(is_correct_format)

In [ ]:
print((tran['date_format']==0).sum())
tran=tran[tran['date_format']!=0]

In [ ]:
#gap on the oct 2021
title="sales per day "
fig=px.histogram(tran,x='date',title=title)
fig.show()

In [ ]:
tmp=tran.groupby('client_id')['quantity_sold'].count().sort_values(ascending=False)

In [ ]:
#let check purchases frequencies
top_clients = tran['client_id'].value_counts().nlargest(20).index
top_tran = tran[tran['client_id'].isin(top_clients)]
title="Number of sales per client (top20)"
fig=px.histogram(top_tran,x='client_id',title=title)
fig.update_xaxes(tickangle=70)
fig.show()

In [ ]:
#get list of top 4 id 
list_big_buyer=tran['client_id'].value_counts()[:4].values
list_big_buyer

In [ ]:
#let check if any T_0 product transaction remains
tran[tran['id_prod']=='T_0']

# dataset diff check before merges

## table client vs transaction

In [ ]:
id_client=set(cust['client_id'])
id_client_achat=set(tran['client_id'])
len(id_client),len(id_client_achat)

In [ ]:
print(id_client_achat-id_client)
print(id_client-id_client_achat)
client_achat_0=list(id_client-id_client_achat)

In [ ]:
tmp=cust[cust['client_id'].isin(client_achat_0)]
len(tmp)

In [ ]:
tmp[tmp['birth']==1980]

In [ ]:
tickvals= list(range(1950,2010,10))
fig=px.histogram(tmp,x='birth',color='sex',barmode='group',title='Distribution of customers with no purchases by gender and date of birth.')
#fig.update_xaxes(type='category')
fig.update_layout(
        xaxis = dict(
                tickmode = 'array',
                tickvals = tickvals,
        )
)
fig.show()

## table products vs transaction

In [ ]:
id_prod=set(products['product_id'])
id_prod_vente=set(tran['id_prod'])
len(id_prod),len(id_prod_vente)

In [ ]:
print(len(id_prod-id_prod_vente),id_prod-id_prod_vente) #23 product without any sales (in given timelaps)
print(id_prod_vente-id_prod) #one transaction has data sa

In [ ]:
### checks no transsactions products
liste_0vente=list(id_prod-id_prod_vente)
tmp=products[products['product_id'].isin(liste_0vente)]
fig=px.histogram(tmp, x='price',color='category',title="Distribution of products without sales records by category")
fig.show()

In [ ]:
tran[tran['id_prod']=='0_2245']
#we have 103 sales of this product, which isnt in our product database, need handling it

In [ ]:
tmp=tran[tran['id_prod']=='0_2245'].copy(deep=True)
tmp['period']=pd.to_datetime(tmp.date).dt.to_period('Q')
tmp['period']=tmp['period'].astype('str')

In [ ]:
fig=px.histogram(tmp.sort_values('period'),x='period',histfunc='count')
fig.update_xaxes(type='category')
fig.show()

In [ ]:
# let check the list of products not having any seles
list_no_sales=list(id_prod-id_prod_vente)
len(list_no_sales) 

In [ ]:
tmp=products[products['product_id'].isin(list_no_sales)]
tmp.head()

In [ ]:
# stock quantity of no sales data  
print(tmp['stock_quantity'].sum())

In [ ]:
px.bar(tmp,x='price',y="stock_quantity",color="category")

# Merges

we do use innner merge, the outliers must be handdle yet

In [ ]:
tmp=pd.merge(tran,products,left_on='id_prod',right_on='product_id')
df=pd.merge(tmp,cust,on='client_id')
df.shape

In [ ]:
df.duplicated().sum()

In [ ]:
df.isnull().sum()

In [ ]:
df.head()

## custom changes before data analysis

In [ ]:
#Conversion de la colonne date: Obj -> datetime64
df['date']=df['date'].astype('datetime64[ns]')

In [ ]:
#Creation d'une colonne extration year_month, on remplace simplement toutes les dates par le 1er du mois
df['monthly'] = df['date'].dt.to_period('M').dt.to_timestamp()
df.head()

In [ ]:
# Age at the purchase time
df['age']=df['date'].dt.year-df.birth
df.head()

In [ ]:
# Binning age
bins = range(0,101,10)
df['ages_bins'] = pd.cut(df['age'], bins=bins)

In [ ]:
mask_client=df['client_id'].isin(['c_6714','c_4958','c_1609','c_3454'])
mask_date=(df['date']<"2021-10-01")|(df['date']>="2021-11-01")
keep_col=['product_id', 'date', 'session_id', 'client_id', 'quantity_sold', 'category', 'price', 'sex', 'birth', 'age', 'ages_bins']
df_livrable=df[(~mask_client) & mask_date][keep_col]

In [ ]:
fig=px.histogram(df_livrable,x='date',color='category')
# fig.update_xaxes(type='category')
fig.show()

# Graphs and Addin

## Corr HeatMap

In [ ]:
corr_df=df_livrable[['price','ages_bins','category','sex']]

In [ ]:
corr_df['category']=corr_df['category'].astype('category').cat.codes
corr_df['sex']=corr_df['sex'].astype('category').cat.codes
corr_df['ages_bins']=corr_df['ages_bins'].astype('category').cat.codes

In [ ]:
plt.figure(figsize=(12, 7))
corr=corr_df.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
heatmap = sns.heatmap(corr, mask=mask,vmin=-1,vmax=1,annot=True)
heatmap.set_title('correction matrix')
plt.xticks(rotation=45,ha='right')
plt.show()

heatmap show some correlation between category of good and price (which is obvious)

## BIG 4
graph of big 4

In [ ]:
big_4=['c_6714','c_4958','c_1609','c_3454']
df_baron=df[df['client_id'].isin(big_4)]

In [ ]:
df_baron

In [ ]:
df_baron.groupby('client_id').agg({'price':"sum","quantity_sold":"sum","category":pd.Series.mode})

In [ ]:
fig=px.histogram(df_baron,x='date',color='client_id')
# fig.update_xaxes(type='category')
fig.show()

In [ ]:
revenu_4=df[df['client_id'].isin(big_4)]['price'].sum()
revenu_other=df[~df['client_id'].isin(big_4)]['price'].sum()
revenu=revenu_4+revenu_other
revenu_other, revenu_4

In [ ]:
px.pie(values=[revenu,revenu_4], names=['total','barons sales'], title='part of revenu by big4')

In [ ]:
title="B2B sales on histogramm"
fig=px.histogram(df_baron, x='client_id',color='category',title=title)
#pio.write_image(fig,Path(path/title),format='png')
fig.show()

In [ ]:
tmp=df_baron.groupby(['client_id','category'])['price'].sum().reset_index()
tmp['category']=tmp['category'].astype('str')
title="Revenu from the B2B clients"
fig=px.bar(tmp, x='client_id',y='price', color='category',title=title)
fig.update_xaxes(categoryarray=['c_1609','c_6714','c_3454','c_4958'])
# pio.write_image(fig,Path(path/title),format='png')
fig.show()

## the GAP on oct 2021

In [ ]:
df[(df['monthly']=='2021-10-01')&(df.category=="Accssoires")]

In [ ]:
title="The Gap on octobre 2021"
fig=px.histogram(df,x='date',color='category',title=title)
# pio.write_image(fig,Path(path/title),format='png')
fig.show()

In [ ]:
# as the graph above could suggest, the "Accesoires" category data is missing for the month
title="Missing data for Accessory category"
fig=px.histogram(df,x='monthly',color='category',title=title)
# pio.write_image(fig,Path(path/title),format='png')
fig.show()

In [ ]:
# détails des ventes pour le mois d'octobre 2021, nous décidons de drop ce mois car il manque des données pour la catégorie 1
tmp=df[(df['date']>"2021-10-01")&(df['date']<="2021-11-01")]
title="Focus on october 2021 seles"
fig=px.histogram(tmp,x='date',color='category',title=title)
#pio.write_image(fig,Path(path/title),format='png')
fig.show()

In [ ]:
df.head()

## KPi Revenue

In [ ]:
# tmp=df_analyse[(df_analyse['date']<pd.to_datetime("2021-10-01"))|(df_analyse['date']>=pd.to_datetime("2021-11-01"))].copy(deep=True)
tmp2=df.groupby('monthly')['price'].sum()
title="Monthly revenue"
fig=px.bar(tmp2,x=tmp2.index,y=tmp2.values,title=title)
#pio.write_image(fig,Path(path/title),format='png')
fig.show()

In [ ]:
tmp=df.groupby(['monthly','category']).agg({"price":'sum'}).reset_index()
tmp['category']=tmp['category'].astype('str')
title='A par mois par catégorie'
fig=px.bar(tmp,x='monthly',y='price',color='category',title='Monthly revenue by category')
fig.show()

In [ ]:
tmp=df.groupby('category')['price'].sum()
fig=px.pie(tmp,values=tmp.values,names=tmp.index,title='Revenue by category')
fig.show()

In [ ]:
df['price'].sum()

## Carts insights

In [ ]:
df_livrable

In [ ]:
tmp.index

In [ ]:
tmp=df_livrable[['price','ages_bins','category']]
fig = px.box(tmp, x=tmp['ages_bins'].astype('str'), y='price', 
              title='Distribution of Cart Amounts by Age Group',
             color="category",
            points=False) 
fig.update_xaxes(categoryorder='category ascending')
fig.show()

In [ ]:
df_livrable.groupby(['age','category'])['price'].sum().reset_index()

In [ ]:
tmp=df_livrable.groupby(['age'])['price'].sum().reset_index()
fig = px.line(tmp, x="age", y="price", title='Expenses by age')
fig.show()

In [ ]:
tmp=df_livrable.groupby(['age','category'])['price'].sum().reset_index()
fig = px.line(tmp, x="age", y="price",color='category', title='Expenses by Age and Category')
fig.show()

## RFM

In [ ]:
#recency data
last=df.date.max().date()
last

In [ ]:
df_recency=df.groupby('client_id',as_index=False)['date'].max()
df_recency['recency']=df_recency["date"].dt.date.apply(lambda x: (last-x).days)
df_recency.head()

In [ ]:
title='recency data by customer'
fig=px.box(df_recency,x='recency',title=title)
#pio.write_image(fig,Path(path/title),format='png')
fig.show()

frequecy and monetary data

In [ ]:
df_frequency=df.groupby('client_id').agg({'session_id':np.size,'price':"sum"}).reset_index()
df_frequency.columns=['client_id','frenquency','monetary']
df_frequency.head()

merge data RFM

In [ ]:
df_rfm=df_recency[['client_id','recency']].merge(df_frequency, on='client_id').set_index('client_id')

In [ ]:
df_rfm.info()

In [ ]:
df_rfm.quantile(q=[0.25,0.5,0.75])

In [ ]:
quantiles=df_rfm.quantile(q=[0.25,0.5,0.75])
quantiles.to_dict()

In [ ]:
# Arguments (x = value, p = recency, monetary_value, frequency, d = quartiles dict)
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

In [ ]:
rfm_segmentation = df_rfm
rfm_segmentation['R_Quartile'] = rfm_segmentation['recency'].apply(RScore, args=('recency',quantiles,))
rfm_segmentation['F_Quartile'] = rfm_segmentation['frenquency'].apply(FMScore, args=('frenquency',quantiles,))
rfm_segmentation['M_Quartile'] = rfm_segmentation['monetary'].apply(FMScore, args=('monetary',quantiles,))

In [ ]:
rfm_segmentation.head()

In [ ]:
rfm_segmentation['RFMScore'] = rfm_segmentation.R_Quartile.map(str) \
                            + rfm_segmentation.F_Quartile.map(str) \
                            + rfm_segmentation.M_Quartile.map(str)
rfm_segmentation.head()

In [ ]:
print("Best Customers: ",len(rfm_segmentation[rfm_segmentation['RFMScore']=='444']))
print('Loyal Customers: ',len(rfm_segmentation[rfm_segmentation['F_Quartile']==4]))
print("Big Spenders: ",len(rfm_segmentation[rfm_segmentation['M_Quartile']==4]))
print('Almost Lost: ', len(rfm_segmentation[rfm_segmentation['RFMScore']=='244']))
print('Lost Customers: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='144']))
print('Lost Cheap Customers: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='111']))